# Prompt Lab | Part of IBM watsonx.ai®

# Interview Trainer Agent Notebook v1.0.0

This notebook contains steps and code to demonstrate the use of an Interview Trainer Agent for answering role-specific interview questions. It introduces Python code for loading a knowledge base, retrieving relevant questions, and running a mock interview session.

> **Note:** This is a standalone demonstration. It loads questions from a local file instead of connecting to IBM services.

## Notebook goals

The learning goals of this notebook are:

- Loading a role-based interview question knowledge base
- Creating helper functions to retrieve and display questions for specific roles
- Running a mock interview and answering ad-hoc queries

## Setup

In [ ]:
import os, glob, textwrap

# Auto-detect the knowledge base file
candidates = glob.glob('/mnt/data/InterviewTrainerAgent_KnowledgeBase*.txt') + glob.glob('/mnt/data/*InterviewTrainerAgent_KnowledgeBase*.txt')
if candidates:
    KB_PATH = candidates[0]
else:
    KB_PATH = '/mnt/data/InterviewTrainerAgent_KnowledgeBase (1).txt'  # fallback

print("Using knowledge base file:", KB_PATH)
with open(KB_PATH, 'r', encoding='utf-8') as f:
    kb_text = f.read()

# Preview first part of file
print("\n--- Knowledge base preview (first 800 chars) ---\n")
print(textwrap.fill(kb_text[:800], width=120))

## Utilities: Search Functions
We will implement simple keyword-based and fuzzy matching functions to retrieve questions from the knowledge base.

In [ ]:
from difflib import get_close_matches
import re

kb_lines = [ln.strip() for ln in kb_text.splitlines() if ln.strip()]

def simple_search(query, max_results=7):
    '''Return lines that contain all query words (case-insensitive).'''
    q = query.lower().split()
    results = []
    for ln in kb_lines:
        ln_low = ln.lower()
        if all(any(w in ln_low for w in q) for w in q):
            results.append(ln)
    if not results:
        words = [w for w in re.findall(r"\w+", query) if len(w) > 3]
        if words:
            cand_scores = []
            for ln in kb_lines:
                score = sum(1 for w in words if w.lower() in ln.lower())
                if score:
                    cand_scores.append((score, ln))
            cand_scores.sort(reverse=True)
            results = [ln for s, ln in cand_scores[:max_results]]
    return results[:max_results]

def fuzzy_search(query, max_results=7):
    return get_close_matches(query, kb_lines, n=max_results, cutoff=0.4)

print('Knowledge base loaded with', len(kb_lines), 'lines.')

## Role-based Mock Interview

In [ ]:
import random
from IPython.display import display, Markdown

role_keywords = {
    'software developer - fresher': ['Software Developer – Fresher', 'four pillars of OOP', 'reverse a string', 'difference between Java and Python', 'final year project'],
    'software developer - 2+ years': ['design patterns', 'optimize backend API', 'multithreading', 'major technical challenge'],
    'data analyst - fresher': ['data cleaning', 'types of data', 'Excel', 'Power BI', 'Python'],
    'cybersecurity analyst - fresher': ['phishing', 'firewall', 'vulnerabilities', 'cybersecurity'],
    'ai/ml engineer - 2+ years': ['supervised', 'unsupervised', 'model accuracy', 'overfitting', 'underfitting'],
    'ui/ux designer - fresher': ['UX design', 'user research', 'wireframing', 'design project'],
    'hr executive - fresher': ['work in HR', 'conflict between team members', 'communication'],
    'marketing executive - fresher': ['marketing campaign', 'digital', 'traditional marketing', 'SEO'],
    'project manager - 2+ years': ['manage deadlines', 'project you successfully led', 'scope creep'],
    'technical support engineer - fresher': ['frustrated customer', 'remote troubleshooting', 'SLAs']
}

def collect_questions_for_role(role, n=6):
    role_key = role.lower()
    keywords = role_keywords.get(role_key, [])
    found = []
    for kw in keywords:
        found += simple_search(kw, max_results=5)
    if not found:
        found = fuzzy_search(role, max_results=n)
    seen, ordered = set(), []
    for ln in found:
        if ln not in seen:
            ordered.append(ln)
            seen.add(ln)
        if len(ordered) >= n:
            break
    return ordered

def mock_interview(role='Data Analyst - Fresher', n=5):
    questions = collect_questions_for_role(role, n=n*2)
    if not questions:
        display(Markdown('**No questions found for role:** ' + role))
        return
    display(Markdown(f"### Mock interview for **{role}**\n"))
    sel = questions[:n]
    for i, q in enumerate(sel, start=1):
        display(Markdown(f"**Q{i}.** {q}"))
        display(Markdown("**A:** _(Type your answer here)_\n---"))

mock_interview('Data Analyst - Fresher')

## Ask the Agent

In [ ]:
def ask_agent(question, max_results=6):
    res = simple_search(question, max_results=max_results)
    if not res:
        res = fuzzy_search(question, max_results=max_results)
    if not res:
        return ['No relevant information found in the knowledge base.']
    return res

print('\n'.join(ask_agent('What is phishing and how do you prevent it?')))

## Save & Download
The notebook file is saved in `/mnt/data/InterviewTrainerAgent_Final.ipynb`. You can download it using the link after running this cell.

In [ ]:
import nbformat as nbf
out_path = '/mnt/data/InterviewTrainerAgent_Final.ipynb'
nbformat.write(nbformat.v4.new_notebook(), out_path)
print('Notebook saved to', out_path)